# Compile YoloV5 model for AWS neuron

* AWS inf1 use aws neuron sdk [link](https://github.com/aws/aws-neuron-sdk)
* AWS Sample code for inf1 pytorch [link](https://github.com/aws/aws-neuron-sdk/blob/master/docs/pytorch-neuron/tutorial-compile-infer.md)
* neuron installation guide [link](https://github.com/aws/aws-neuron-sdk/blob/master/docs/neuron-install-guide.md)
* can't compile model. file an [issue](https://github.com/aws/aws-neuron-sdk/issues/141)

In [ ]:
!pip install tensorboard onnx torch torchvision --upgrade

In [1]:
!pip install neuron-cc torch-neuron --extra-index-url="https://pip.repos.neuron.amazonaws.com"

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Using cached https://pip.repos.neuron.amazonaws.com/neuron-cc/neuron_cc-1.0.16861.0%2B6021080341-cp36-cp36m-linux_x86_64.whl (45.2 MB)
  Using cached https://pip.repos.neuron.amazonaws.com/torch-neuron/torch_neuron-1.0.1386.0-py3-none-any.whl (5.1 MB)
  Using cached https://pip.repos.neuron.amazonaws.com/dmlc-nnvm/dmlc_nnvm-1.0.2732.0%2B6020726378-py3-none-any.whl (14.8 MB)
  Using cached https://pip.repos.neuron.amazonaws.com/islpy/islpy-2018.2%2Baws2018.x.456.0.bld6017592525-cp36-cp36m-linux_x86_64.whl (1.6 MB)
  Using cached https://pip.repos.neuron.amazonaws.com/dmlc-tvm/dmlc_tvm-1.0.2732.0%2B6020726378-cp36-cp36m-linux_x86_64.whl (72.8 MB)
  Using cached scipy-1.3.2-cp36-cp36m-manylinux1_x86_64.whl (25.2 MB)
  Using cached https://pip.repos.neuron.amazonaws.com/dmlc-topi/dmlc_topi-1.0.2732.0%2B6020726378-py3-none-any.whl (1.5 MB)
  Using cached https://pip.repos.neuron.amazonaws.com/inferentia-hw

In [2]:
!git clone https://github.com/ultralytics/yolov5

fatal: destination path 'yolov5' already exists and is not an empty directory.


In [3]:
import sys, os, onnx, torch
import torch_neuron

fp_yolov5 = os.path.abspath(os.getcwd()) + "/yolov5"
sys.path.append(fp_yolov5)
sys.path

['',
 '/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python36.zip',
 '/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6',
 '/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/lib-dynload',
 '/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages',
 '/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/IPython/extensions',
 '/home/ec2-user/.ipython',
 '/home/ec2-user/SageMaker/myAWSStudyBlog/yolov5-onnx-sm/6-yolov5-inf1/yolov5']

In [4]:
%cd ./yolov5

/home/ec2-user/SageMaker/myAWSStudyBlog/yolov5-onnx-sm/6-yolov5-inf1/yolov5


In [5]:
!python -c "from utils.google_utils import *; attempt_download('weights/yolov5s.pt');"

In [10]:
img = torch.zeros((1, 3, 640, 640))
fp_model = './weights/yolov5s.pt'
model = torch.load(fp_model, map_location=torch.device('cpu'))['model'].float()
model.eval()
model.model[-1].export = False
y = model(img)  # dry run to inference mode

model_neuron = torch.neuron.trace(model, example_inputs=[img])
model_neuron.save("yolov5_neuron.pt")

TypeError: copy_() missing 1 required positional arguments: "other"

In [ ]:
torch.__version__